### 问题:
我们想实现一个状态机，或者让对象可以在不同的状态中进行操作。但是我们并不希
望代码里会因此出现大量的条件判断。

##### 解决方案:
在某些应用程序中，我们可能会让对象根据某种内部状态来进行不同的操作。例如，
考虑下面这个代表网络连接的类：

In [13]:
class Connection:
    def __init__(self):
        self.state = 'CLOSED'
    def read(self):
        if self.state != 'OPEN':
            raise RuntimeError('Not open')
        print('reading')
    def write(self, data):
        if self.state != 'OPEN':
            raise RuntimeError('Not open')
        print('writing')
    def open(self):
        if self.state == 'OPEN':
            raise RuntimeError('Already open')
        self.state = 'OPEN'
    def close(self):
        if self.state == 'CLOSED':
            raise RuntimeError('Already closed')
        self.state = 'CLOSED'

这份代码为我们提出了几个难题。首先，由于代码中引入了许多针对状态的条件检查，
代码变得很复杂。其次，程序的性能下降了。因为普通的操作如读（read()）和写（write()）
总是要在处理前先检查状态。

一个更加优雅的方式是将每种操作状态以一个单独的类来定义，然后在 Connection 类
中使用这些状态类。示例如下：

In [14]:
class Connection:
    def __init__(self):
        self.new_state(ClosedConnectionState)  #此处传入一个类的地址
    def new_state(self, newstate):
        self._state = newstate  #类似实例化一个类
    # Delegate to the state class
    def read(self):
        return self._state.read(self)
    def write(self, data):
        return self._state.write(self, data)
    def open(self):
        return self._state.open(self)
    def close(self):
        return self._state.close(self)

# Connection state base class
class ConnectionState:                          #基类
    @staticmethod
    def read(conn):
        raise NotImplementedError()
    @staticmethod
    def write(conn, data):
        raise NotImplementedError()
    @staticmethod
    def open(conn):
        raise NotImplementedError()
    @staticmethod
    def close(conn):
        raise NotImplementedError()
        
# Implementation of different states        
class ClosedConnectionState(ConnectionState):   #close类
    @staticmethod
    def read(conn):
        raise RuntimeError('Not open')
    @staticmethod
    def write(conn, data):
        raise RuntimeError('Not open')
    @staticmethod
    def open(conn):
        conn.new_state(OpenConnectionState)   #切换为open类
    @staticmethod
    def close(conn):
        raise RuntimeError('Already closed')

class OpenConnectionState(ConnectionState):     #open类  
    @staticmethod
    def read(conn):
        print('reading')
    @staticmethod
    def write(conn, data):
        print('writing')
    @staticmethod
    def open(conn):
        raise RuntimeError('Already open')
    @staticmethod
    def close(conn):
        conn.new_state(ClosedConnectionState) 

c = Connection()
print(c._state)
c.open()
print(c._state)
c.read()
c.write('hello')
c.close()
print(c._state) 

<class '__main__.ClosedConnectionState'>
<class '__main__.OpenConnectionState'>
reading
writing
<class '__main__.ClosedConnectionState'>


编写含有大量复杂的条件判断并和各种状态纠缠在一起的代码是难以维护和解读的。
本节给出的解决方案通过将各个状态分解为单独的类来避免这个问题。

可能看起来有些奇怪，这里每种状态都用类和静态方法来实现，在每个静态方法中都
把 Connection 类的实例作为第一个参数。产生这种设计的原因在于我们决定在不同的
状态类中不保存任何实例数据。相反，所有的实例数据应该保存在 Connection 实例中。
将所有的状态放在一个公共的基类下，这么做的大部分原因是为了帮助组织代码，并
确保适当的方法得到了实现。在基类方法中出现的 NotImplementedError 异常是为了确
保在子类中实现了所需的方法。作为替代方案，可以考虑使用 8.12 节中描述过的抽象
基类。

另一种实现方法是考虑去直接修改实例的__class__属性。示例如下：

In [15]:
class Connection:
    def __init__(self):
        self.new_state(ClosedConnection)
    def new_state(self, newstate):
        self.__class__ = newstate
    def read(self):
        raise NotImplementedError()
    def write(self, data):
        raise NotImplementedError()
    def open(self):
        raise NotImplementedError()
    def close(self):
        raise NotImplementedError()

class ClosedConnection(Connection):
    def read(self):
        raise RuntimeError('Not open')
    def write(self, data):
        raise RuntimeError('Not open')
    def open(self):
        self.new_state(OpenConnection)
    def close(self):
        raise RuntimeError('Already closed')
    def state(self):
        return self.__class__

class OpenConnection(Connection):
    def read(self):
        print('reading')
    def write(self, data):
        print('writing')
    def open(self):
        raise RuntimeError('Already open')  
    def close(self):
        self.new_state(ClosedConnection)
    def state(self):
        return self.__class__

c=Connection()
c.open()
c.write('hello')
c.close()


writing


这种实现方法的主要特点就是消除了额外的间接关系。这里不再将 Connection 和
ConnectionState 作为单独的类来实现，现在我们将它们合并在一起了。随着状态的改
变，实例也会修改自己的类型。示例如下：

In [16]:
c = Connection() 
print(c.state())
c.open()
print(c.state())
c.close()
print(c.state())

<class '__main__.ClosedConnection'>
<class '__main__.OpenConnection'>
<class '__main__.ClosedConnection'>


面向对象编程的拥趸不喜欢这种直接修改实例的__class__属性的做法。但是在技术上
是允许这么做的。此外，这么做也会让代码的执行速度更快些，因为现在调用 connection
上的所有方法都不必再经过一层额外的间接步骤了。

最后，无论上面哪种技术对于实现复杂的状态机都是很有用的——尤其是在那些可能
出现大量的 if-elif-else 块的代码中。